###**Detecção de Fraudes no IEEE-CIS Fraud Detection com LSTM no PyTorch**

**Objective:** Optimize a pre-trained neural network model for credit card fraud detection. Apply advanced hyperparameter fine-tuning techniques, such as grid search and random search, to improve model performance metrics, including precision, recall, F1-score, and AUC-ROC. The activity also requires a comparison between the optimized model and the original model, allowing the assessment of the impact of hyperparameter modifications on overall performance.

**TLDR;**

The pipeline was sound from an engineering standpoint (temporal splitting, scaler, callbacks), but the global sequence hypothesis doesn't hold for this dataset—and the class is extremely rare. The final result (AUC ≈ 0.52; very low F1) indicates that the LSTM, as the sequences were formed, didn't learn a useful signal. Restructuring the problem by entity or migrating to a tabular model with good features should yield large and immediate gains; then, adjust the threshold and costs to achieve the desired recall without causing false positives.

## Importações



In [2]:
%pip install gdown
import gdown
import os
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)
from sklearn.utils.class_weight import compute_class_weight
from scipy.stats import pearsonr

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


## Configurações

In [3]:
arquivo_destino_colab = "dataset.csv"
doc_id = "1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl"
URL = f"https://drive.google.com/uc?id={doc_id}"
gdown.download(URL, arquivo_destino_colab, quiet=False)


SAVE_FIGS = False
LOOKBACK = 10
VAL_FRACTION = 0.1
TEST_FRACTION = 0.2
BATCH_SIZE = 512
EPOCHS = 30
LEARNING_RATE = 1e-3
NEGATIVE_TO_POS_RATIO_TRAIN = 10
SEED = 42

# Fixar sementes para reprodutibilidade
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seeds(SEED)

# Pasta para figuras
if SAVE_FIGS:
    os.makedirs("figs", exist_ok=True)

Downloading...
From (original): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl
From (redirected): https://drive.google.com/uc?id=1u_OWAPkIdgJw1ah5xP_dGBFMSANxjxEl&confirm=t&uuid=dcddb62d-0abe-458d-9168-3e42b6dff18c
To: /content/dataset.csv
100%|██████████| 151M/151M [00:02<00:00, 61.0MB/s]


'dataset.csv'

In [4]:
df = pd.read_csv("dataset.csv")
df.head()

## Utils

In [14]:
def safe_log1p(series: pd.Series) -> pd.Series:
    """Aplica log1p em valores não negativos; se houver negativos, desloca."""
    if (series < 0).any():
        shift = abs(series.min()) + 1e-9
        return np.log1p(series + shift)
    return np.log1p(series)

def print_header(title: str):
    print("\n" + "="*len(title))
    print(title)
    print("="*len(title))

def describe_correlation_with_class(df: pd.DataFrame, feature_cols, target_col="Class", top_k=10):
    """Correlação ponto-biserial (Pearson com rótulo binário) por feature."""
    corrs = []
    for c in feature_cols:
        try:
            r, p = pearsonr(df[c].values, df[target_col].values)
            corrs.append((c, r, p))
        except Exception:
            corrs.append((c, np.nan, np.nan))
    corr_df = pd.DataFrame(corrs, columns=["feature", "r_point_biserial", "p_value"]).sort_values(
        by="r_point_biserial", key=lambda s: s.abs(), ascending=False
    )
    print_header("Top correlações (|r|) com Class")
    print(corr_df.head(top_k).to_string(index=False))
    return corr_df

def chronological_split(df: pd.DataFrame, time_col="Time", test_fraction=0.2):
    """Divide cronologicamente: parte mais recente vira teste."""
    df_sorted = df.sort_values(time_col).reset_index(drop=True)
    split_idx = int(len(df_sorted) * (1 - test_fraction))
    train_val_df = df_sorted.iloc[:split_idx].reset_index(drop=True)
    test_df = df_sorted.iloc[split_idx:].reset_index(drop=True)
    return train_val_df, test_df

def build_sequences(X: np.ndarray, y: np.ndarray, lookback: int):
    """Cria sequências deslizantes X_seq (n_samples, lookback, n_feat) e targets y_seq (n_samples,)."""
    if lookback < 1:
        raise ValueError("lookback deve ser >= 1")
    Xs, ys = [], []
    for i in range(lookback, len(X)):
        Xs.append(X[i - lookback:i, :])
        ys.append(y[i])
    return np.array(Xs, dtype=np.float32), np.array(ys, dtype=np.int32)

def undersample_negatives(X, y, max_ratio=10, seed=42):
    """Mantém todos os positivos e reduz negativos para ~max_ratio:1 (no conjunto de TREINO)."""
    pos_idx = np.where(y == 1)[0]
    neg_idx = np.where(y == 0)[0]
    if len(pos_idx) == 0:
        return X, y  # nada a fazer
    max_neg = min(len(neg_idx), max_ratio * len(pos_idx))
    rng = np.random.default_rng(seed)
    selected_neg = rng.choice(neg_idx, size=max_neg, replace=False) if len(neg_idx) > max_neg else neg_idx
    keep = np.concatenate([pos_idx, selected_neg])
    rng.shuffle(keep)
    return X[keep], y[keep]

def best_threshold_from_validation(y_true_val, y_score_val):
    """Escolhe threshold que maximiza F1 na validação."""
    thresholds = np.linspace(0.05, 0.95, 19)
    best_t = 0.5
    best_f1 = -1
    for t in thresholds:
        y_pred = (y_score_val >= t).astype(int)
        f1 = f1_score(y_true_val, y_pred, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    return best_t, best_f1

def build_lstm_model(n_features: int, lookback: int, lr=1e-3) -> tf.keras.Model:
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=(lookback, n_features)),
        BatchNormalization(),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dense(1, activation="sigmoid")
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="binary_crossentropy",
        metrics=[
            tf.keras.metrics.AUC(curve="ROC", name="auc"),
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall")
        ]
    )
    return model

## EDA - Análise Exploratória



In [18]:
print_header("Visão geral")
print(df.head(5))
print("\nInfo:")
print(df.info())

print_header("Valores ausentes por coluna")
print(df.isna().sum().sort_values(ascending=False))

print_header("Estatísticas descritivas (numéricas)")
print(df.describe(percentiles=[.01, .05, .25, .5, .75, .95, .99]).T)

# Distribuição das classes (desbalanceamento)
print_header("Distribuição de Classes")
class_counts = df["Class"].value_counts().sort_index()
class_pct = 100 * class_counts / len(df)
print(pd.DataFrame({"count": class_counts, "pct_%": class_pct.round(4)}))

fig, ax = plt.subplots(figsize=(4,3))
sns.countplot(x="Class", data=df, ax=ax)
ax.set_title("Distribuição da variável alvo (Class)")
plt.close(fig)

# Distribuições de Amount (linear e log)
fig, axes = plt.subplots(1, 2, figsize=(10,3))
sns.histplot(df["Amount"], bins=100, ax=axes[0])
axes[0].set_title("Amount (linear)")

sns.histplot(safe_log1p(df["Amount"]), bins=100, ax=axes[1])
axes[1].set_title("log1p(Amount)")
plt.close(fig)

# Amount por classe
fig, ax = plt.subplots(figsize=(5,3))
sns.boxplot(x="Class", y="Amount", data=df, ax=ax, showfliers=True)
ax.set_title("Amount por Class (outliers visíveis)")
plt.close(fig)

# Correlação entre features e Class (ponto-biserial)
feature_cols = [c for c in df.columns if c not in ["Class"]]
corr_df = describe_correlation_with_class(df, feature_cols, target_col="Class", top_k=12)

# Heatmap de correlação entre features (Pearson)
fig, ax = plt.subplots(figsize=(10,8))
corr_mat = df.drop(columns=["Class"]).corr()
sns.heatmap(corr_mat, cmap="vlag", center=0, ax=ax)
ax.set_title("Matriz de correlações (features)")
plt.close(fig)


Visão geral
   Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V

## Preparação dos dados para LSTM


In [19]:
print_header("Preparação dos dados para LSTM (split temporal, escalonamento, janelas)")

# Ordena por tempo e divide TREINO/TESTE cronologicamente (evita vazamento futuro->passado)
train_val_df, test_df = chronological_split(df, time_col="Time", test_fraction=TEST_FRACTION)
print(f"Train+Val shape: {train_val_df.shape} | Test shape: {test_df.shape}")

# Dentro do TREINO, separamos uma validação temporal (parte final do treino)
split_idx_tv = int(len(train_val_df) * (1 - VAL_FRACTION))
train_df = train_val_df.iloc[:split_idx_tv].reset_index(drop=True)
val_df = train_val_df.iloc[split_idx_tv:].reset_index(drop=True)

print(f"Train shape: {train_df.shape} | Val shape: {val_df.shape} | Test shape: {test_df.shape}")

# Trata nulos (se houver). Aqui: imputação simples com mediana.
for col in feature_cols:
    med = train_df[col].median()
    train_df[col] = train_df[col].fillna(med)
    val_df[col]   = val_df[col].fillna(med)
    test_df[col]  = test_df[col].fillna(med)

# Escalonamento (fit no TREINO, aplica em VAL/TEST) — StandardScaler é ok para PCA e Amount
scaler = StandardScaler()
scaler.fit(train_df[feature_cols].values)

X_train_all = scaler.transform(train_df[feature_cols].values)
y_train_all = train_df["Class"].values.astype(int)

X_val_all = scaler.transform(val_df[feature_cols].values)
y_val_all = val_df["Class"].values.astype(int)

X_test_all = scaler.transform(test_df[feature_cols].values)
y_test_all = test_df["Class"].values.astype(int)

# Criação de sequências (deslizante, etiqueta = último passo)
X_train_seq, y_train_seq = build_sequences(X_train_all, y_train_all, LOOKBACK)
X_val_seq,   y_val_seq   = build_sequences(X_val_all, y_val_all, LOOKBACK)
X_test_seq,  y_test_seq  = build_sequences(X_test_all, y_test_all, LOOKBACK)

print(f"Seq shapes -> X_train: {X_train_seq.shape}, X_val: {X_val_seq.shape}, X_test: {X_test_seq.shape}")

# (Opcional) Undersampling de negativos no TREINO para acelerar e melhorar aprendizado inicial
X_train_seq_bal, y_train_seq_bal = undersample_negatives(
    X_train_seq, y_train_seq, max_ratio=NEGATIVE_TO_POS_RATIO_TRAIN, seed=SEED
)
print(f"Após undersampling  -> X_train: {X_train_seq_bal.shape}, positivos: {y_train_seq_bal.sum()}, "
      f"negativos: {len(y_train_seq_bal) - y_train_seq_bal.sum()}")

# Pesos de classe (baseados no conjunto de treino balanceado/atual)
classes = np.array([0, 1])
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train_seq_bal
)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}
print(f"Class weights: {class_weight_dict}")


Preparação dos dados para LSTM (split temporal, escalonamento, janelas)
Train+Val shape: (227845, 31) | Test shape: (56962, 31)
Train shape: (205060, 31) | Val shape: (22785, 31) | Test shape: (56962, 31)
Seq shapes -> X_train: (205050, 10, 30), X_val: (22775, 10, 30), X_test: (56952, 10, 30)
Após undersampling  -> X_train: (4323, 10, 30), positivos: 393, negativos: 3930
Class weights: {0: np.float64(0.55), 1: np.float64(5.5)}


## Modelo LSTM

In [22]:
print_header("Construindo e treinando o modelo LSTM")

n_features = X_train_seq.shape[-1]
model = build_lstm_model(n_features=n_features, lookback=LOOKBACK, lr=LEARNING_RATE)
model.summary()

callbacks = [
    EarlyStopping(monitor="val_auc", mode="max", patience=5, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_auc", mode="max", factor=0.5, patience=3, min_lr=1e-6, verbose=1)
]

history = model.fit(
    X_train_seq_bal, y_train_seq_bal,
    validation_data=(X_val_seq, y_val_seq),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    class_weight=class_weight_dict,
    callbacks=callbacks,
    verbose=2
)

# Curvas de aprendizado (loss e AUC)
fig, ax = plt.subplots(figsize=(6,4))
ax.plot(history.history["loss"], label="train_loss")
ax.plot(history.history["val_loss"], label="val_loss")
ax.set_title("Curva de aprendizado - Loss")
ax.set_xlabel("Época"); ax.set_ylabel("Loss"); ax.legend()
plt.close(fig)

fig, ax = plt.subplots(figsize=(6,4))
ax.plot(history.history["auc"], label="train_auc")
ax.plot(history.history["val_auc"], label="val_auc")
ax.set_title("Curva de aprendizado - AUC")
ax.set_xlabel("Época"); ax.set_ylabel("AUC"); ax.legend()
plt.close(fig)


Construindo e treinando o modelo LSTM


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ (None, 10, 64)         │        24,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 10, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 10, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,537 (146.63 KB)

 Trainable params: 37,409 (146.13 KB)

 Non-trainable params: 128 (512.00 B)

Epoch 1/30
9/9 - 7s - 783ms/step - auc: 0.5761 - loss: 0.6879 - precision: 0.0973 - recall: 0.8321 - val_auc: 0.5248 - val_loss: 0.6515 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-03
Epoch 2/30
9/9 - 2s - 226ms/step - auc: 0.6750 - loss: 0.6555 - precision: 0.1186 - recall: 0.7710 - val_auc: 0.5194 - val_loss: 0.5908 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-03
Epoch 3/30
9/9 - 2s - 248ms/step - auc: 0.7003 - loss: 0.6373 - precision: 0.1431 - recall: 0.7023 - val_auc: 0.5263 - val_loss: 0.5328 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-03
Epoch 4/30
9/9 - 3s - 358ms/step - auc: 0.7177 - loss: 0.6229 - precision: 0.1622 - recall: 0.6361 - val_auc: 0.5273 - val_loss: 0.4860 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-03
Epoch 5/30
9/9 - 4s - 407ms/step - auc: 0.7316 - loss: 0.6140 - precision: 0.1731 - recall: 0.6463 - val_auc: 0.5362 - val_loss: 0.4

## Avaliação (threshold otimizado na validação)

In [23]:
print_header("Avaliação no conjunto de teste")

# Probabilidades
y_val_scores = model.predict(X_val_seq, batch_size=BATCH_SIZE).ravel()
y_test_scores = model.predict(X_test_seq, batch_size=BATCH_SIZE).ravel()

# Escolher threshold via F1 na validação
best_t, best_f1_val = best_threshold_from_validation(y_val_seq, y_val_scores)
print(f"Threshold ótimo (val) p/ F1: {best_t:.3f} | F1(val)={best_f1_val:.4f}")

# Predições binarias no teste
y_test_pred = (y_test_scores >= best_t).astype(int)

# Métricas no teste
acc  = accuracy_score(y_test_seq, y_test_pred)
prec = precision_score(y_test_seq, y_test_pred, zero_division=0)
rec  = recall_score(y_test_seq, y_test_pred, zero_division=0)
f1   = f1_score(y_test_seq, y_test_pred, zero_division=0)
auc  = roc_auc_score(y_test_seq, y_test_scores)

print_header("Métricas no Teste")
print(f"Accuracy : {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall   : {rec:.4f}")
print(f"F1-score : {f1:.4f}")
print(f"AUC-ROC  : {auc:.4f}")

print("\nClassification Report (Teste):")
print(classification_report(y_test_seq, y_test_pred, digits=4, zero_division=0))

# Matriz de confusão
cm = confusion_matrix(y_test_seq, y_test_pred)
fig, ax = plt.subplots(figsize=(4,3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax)
ax.set_title("Matriz de Confusão (Teste)")
ax.set_xlabel("Predito"); ax.set_ylabel("Real")
plt.close(fig)

# Curva ROC
fpr, tpr, _ = roc_curve(y_test_seq, y_test_scores)
fig, ax = plt.subplots(figsize=(5,4))
ax.plot(fpr, tpr, label=f"ROC (AUC={auc:.3f})")
ax.plot([0,1],[0,1],"--", label="Aleatório")
ax.set_title("Curva ROC (Teste)")
ax.set_xlabel("FPR"); ax.set_ylabel("TPR"); ax.legend()
plt.close(fig)


Avaliação no conjunto de teste
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 28ms/step
112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step
Threshold ótimo (val) p/ F1: 0.400 | F1(val)=0.0071

Métricas no Teste
Accuracy : 0.9926
Precision: 0.0028
Recall   : 0.0133
F1-score : 0.0047
AUC-ROC  : 0.5186

Classification Report (Teste):
              precision    recall  f1-score   support

           0     0.9987    0.9938    0.9963     56877
           1     0.0028    0.0133    0.0047        75

    accuracy                         0.9926     56952
   macro avg     0.5008    0.5036    0.5005     56952
weighted avg     0.9974    0.9926    0.9950     56952



## Diagnóstico: over/underfitting (heurístico) e discussão

In [25]:
print_header("Diagnóstico")

train_loss_min = np.min(history.history["loss"])
val_loss_min   = np.min(history.history["val_loss"])
train_auc_max  = np.max(history.history["auc"])
val_auc_max    = np.max(history.history["val_auc"])

print(f"Menor loss - treino: {train_loss_min:.4f} | validação: {val_loss_min:.4f}")
print(f"Maior AUC  - treino: {train_auc_max:.4f} | validação: {val_auc_max:.4f}")


Diagnóstico
Menor loss - treino: 0.4816 | validação: 0.1608
Maior AUC  - treino: 0.8578 | validação: 0.5877


## Análise Geral

**EDA**

* Sem nulos em nenhuma coluna (284.807 linhas, 31 colunas). 👍
* Desbalanceamento extremo: só 0,17% de fraudes (492 casos). Isso explica por que accuracy alto pode enganar.
* Amount é altamente assimétrico (máx. 25.691), típico de transações.
* Maior associação com fraude (ponto-biserial): V17, V14, V12, V10… todas com correlações moderadas e negativas (ex.: V17 ≈ -0,33). Isso sugere que as combinações PCA capturam padrões úteis, mas não fortíssimos.

**Preparação — escolhas e impactos**

* Split temporal (train/val/test = 205k/22,8k/56,9k) — correto para simular produção e evitar vazamento futuro→passado.
* Escalonamento com StandardScaler apenas no treino — correto.
* Janelas LSTM (lookback=10) montadas na sequência global de transações. Sem um ID de entidade (cartão/cliente), a sequência “emenda” usuários diferentes; isso dilui o sinal temporal que a LSTM precisa.
* Undersampling do treino para ~10:1 (neg:pos) e class weights (0: 0,55; 1: 5,5).

**Treino**

* Arquitetura LSTM leve (~38k parâmetros) + BN/Dropout + EarlyStopping/LR scheduler — tecnicamente ok.
* AUC (treino) subiu até ~0,86, mas AUC (val) ficou em ~0,59 no melhor ponto (época 11).
* * As métricas Keras de val_precision/val_recall zeradas por várias épocas revelam que, no threshold 0,5, o modelo quase não marca positivos na validação (com base no output sigmoide).
Val_loss ≪ train_loss não significa “ótimo”: o loss de treino é ponderado por classe, o de validação não — isso sozinho pode deixar o val_loss artificialmente menor.

**Avaliação**

* Threshold otimizado via F1 na validação: 0,40, mas ainda F1(val) = 0,007 (PR fraca).

Teste:

* AUC-ROC = 0,519 → praticamente aleatório.
* Accuracy = 0,993 (enganoso pelo desbalanceamento).
* Precision = 0,0028, Recall = 0,0133, F1 = 0,0047.
Em números humanos: de 75 fraudes no teste, o modelo pegou ~1; e entre centenas de alertas, quase todos são falsos.

**Diagnóstico**

* Gap treino vs validação (AUC 0,86 vs 0,59) sugere overfitting ao treino balanceado/ponderado.
* Sinal temporal pobre para LSTM: sem sequências por entidade, a rede aprende “ruído global” em vez de padrões de comportamento do mesmo cartão/usuário.
* Métrica inadequada durante o treino: accuracy/precision/recall com threshold fixo de 0,5 pouco ajudaram; AUC até ajuda, mas em classes raras PR-AUC é mais informativa.
* Undersampling + class weights no treino, mas val/test reais extremamente desbalanceados → calibração ruim e degradação de PR.

**O que eu mudaria (possíveis próximos passos)**

1. Se houver ID de entidade, reconstruir o problema como sequências por cartão/cliente, respeitando fronteiras entre entidades nos splits e nas janelas. Isso muda o jogo para LSTM/GRU.
2. Baseline forte tabular (sem sequência): LogisticRegression/XGBoost com features simples (Amount log, horário do dia, idade da conta, contagens/estatísticas rolling por entidade, interações básicas). Em geral, no Credit 3. 3. Card Fraud clássico, modelos tabulares performam muito bem.
Métricas focadas em raridade: acompanhar PR-AUC, Recall@K e otimização de threshold por custo (FN vs FP).
4. Reamostrar com cautela: manter val/test com a distribuição real; no treino, testar focal loss ou apenas class weights (sem undersampling agressivo) para melhorar a calibração.
5. Engenharia de atributos temporais (se houver entidade): deltas entre transações, contagens no último T minutos, média/mediana/quantis por janela, frequência por comerciante, etc.
6. Regularização & simplicidade: se insistir em LSTM, reduzir camadas/neurônios ou aumentar Dropout; e aumentar o lookback só faz sentido se a sequência por mesma entidade existir.
7. Calibração de probabilidades (Platt/Isotonic) após o treino para thresholds estáveis.
